Importar librerias

In [2]:
!pip install biopython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from Bio import AlignIO



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_fasta = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/aligned_cluster_sequences_01.fasta"

Modelo de Markov con la librería HMM

Construcción de matrices de transición y emisión

In [ ]:
from collections import defaultdict
import math

In [ ]:
# Cargar el alineamiento desde FASTA
alignment = AlignIO.read("/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/aligned_cluster_sequences_01.fasta", "fasta")
sequences = [str(record.seq).upper() for record in alignment]
num_seqs = len(sequences)
seq_len = len(sequences[0])

# Paso 1: Detectar columnas MATCH
def detectar_columnas_match(sequences, umbral_gap=0.5):
    match_cols = []
    for i in range(len(sequences[0])):
        gaps = sum(1 for seq in sequences if seq[i] == '-')
        match = (gaps / len(sequences)) < umbral_gap
        match_cols.append(match)
    return match_cols

match_columns = detectar_columnas_match(sequences)

# Paso 2: Contar transiciones y emisiones
transiciones = defaultdict(lambda: defaultdict(int))
emisiones = defaultdict(lambda: defaultdict(int))

for seq in sequences:
    prev_state = "START"
    match_idx = 1

    for i, res in enumerate(seq):
        if match_columns[i]:  # Columna tipo Match o Delete
            if res == "-":
                state = f"D{match_idx}"
            else:
                state = f"M{match_idx}"
                emisiones[state][res] += 1
            transiciones[prev_state][state] += 1
            prev_state = state
            match_idx += 1
        else:  # Columna tipo Insert
            if res != "-":
                state = f"I{match_idx - 1}"
                emisiones[state][res] += 1
                transiciones[prev_state][state] += 1
                prev_state = state

    transiciones[prev_state]["END"] += 1

# Paso 3: Normalizar a log-probabilidades
def normalizar_log_probabilidades(conteos, pseudocuento=1e-6):
    log_probs = {}
    for estado, destinos in conteos.items():
        total = sum(destinos.values()) + pseudocuento * len(destinos)
        log_probs[estado] = {
            s: math.log((v + pseudocuento) / total)
            for s, v in destinos.items()
        }
    return log_probs

log_transiciones = normalizar_log_probabilidades(transiciones)
log_emisiones = normalizar_log_probabilidades(emisiones)

# Mostrar resumen de estados y transiciones/emisiones iniciales
{
    "num_estados_match": sum(match_columns),
    "estados_transicion": list(log_transiciones.keys())[:5],
    "emisiones_M1": log_emisiones.get("M1", {}),
    "transiciones_M1": log_transiciones.get("M1", {})
}

{'num_estados_match': 45,
 'estados_transicion': ['START', 'D1', 'D2', 'D3', 'D4'],
 'emisiones_M1': {'M': -0.03636765128727696,
  'L': -4.153185024647633,
  'X': -7.244226523460993,
  'A': -5.857932912340633,
  'C': -5.6347894110264125,
  'F': -5.857932912340633,
  'S': -5.6347894110264125,
  'K': -6.145614901459106,
  'T': -6.5510798429006725,
  'V': -6.145614901459106,
  'P': -7.244226523460993},
 'transiciones_M1': {'M2': -0.0021451570591398735, 'D2': -6.145614895030534}}

In [ ]:
# Algoritmo de Viterbi en log-space
def viterbi(sequence, log_transitions, log_emissions, match_columns):
    states = sorted(log_emissions.keys(), key=lambda x: (x[0], int(x[1:])))
    seq_len = len(sequence)
    n_states = len(states)

    # Inicialización
    V = np.full((n_states, seq_len + 1), -np.inf)
    backpointer = np.zeros((n_states, seq_len + 1), dtype=int)

    state_index = {state: idx for idx, state in enumerate(states)}

    # Estados que pueden seguir a START
    for state in states:
        if "START" in log_transitions and state in log_transitions["START"]:
            emis = log_emissions[state].get(sequence[0], -np.inf)
            V[state_index[state], 1] = log_transitions["START"][state] + emis

    # Recursión
    for t in range(2, seq_len + 1):
        for curr_state in states:
            emis = log_emissions[curr_state].get(sequence[t - 1], -np.inf)
            max_prob = -np.inf
            max_state = 0
            for prev_state in states:
                if prev_state in log_transitions and curr_state in log_transitions[prev_state]:
                    prob = V[state_index[prev_state], t - 1] + log_transitions[prev_state][curr_state] + emis
                    if prob > max_prob:
                        max_prob = prob
                        max_state = state_index[prev_state]
            V[state_index[curr_state], t] = max_prob
            backpointer[state_index[curr_state], t] = max_state

    # Terminación
    best_path_prob = -np.inf
    best_last_state = 0
    for state in states:
        if state in log_transitions and "END" in log_transitions[state]:
            prob = V[state_index[state], seq_len] + log_transitions[state]["END"]
            if prob > best_path_prob:
                best_path_prob = prob
                best_last_state = state_index[state]

    # Backtracking
    best_path = [best_last_state]
    for t in range(seq_len, 1, -1):
        best_state = backpointer[best_path[0], t]
        best_path.insert(0, best_state)

    best_state_path = [states[i] for i in best_path]

    return best_state_path, best_path_prob

# Prueba con una secuencia ejemplo
test_sequence = "MCCGKCLRKCG"
sequence_clean = [aa for aa in test_sequence if aa != '-']
best_path, score = viterbi(sequence_clean, log_transiciones, log_emisiones, match_columns)

best_path, score

(['I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0'], -inf)

In [ ]:
log_emissions.get("I0", {})
log_emissions.get("M1", {})

{'M': -0.03636765128727696,
 'L': -4.153185024647633,
 'X': -7.244226523460993,
 'A': -5.857932912340633,
 'C': -5.6347894110264125,
 'F': -5.857932912340633,
 'S': -5.6347894110264125,
 'K': -6.145614901459106,
 'T': -6.5510798429006725,
 'V': -6.145614901459106,
 'P': -7.244226523460993}

In [ ]:
alignment = AlignIO.read(file_fasta, "fasta")

# Convertimos el alineamiento a una lista de strings
aligned_seqs = [str(rec.seq) for rec in alignment]
print(f"Número de secuencias: {len(aligned_seqs)}")
print(f"Largo del alineamiento: {len(aligned_seqs[0])}")

Número de secuencias: 2552
Largo del alineamiento: 1129


In [ ]:
sequences = [str(record.seq) for record in alignment]
num_seqs = len(sequences)
align_len = len(sequences[0])

# Identificar columnas como MATCH (menos de 50% gaps)
def detect_match_columns(sequences, threshold=0.5):
    match_cols = []
    for col in range(len(sequences[0])):
        gap_count = sum(1 for seq in sequences if seq[col] == '-')
        is_match = (gap_count / len(sequences)) < threshold
        match_cols.append(is_match)
    return match_cols

match_cols = detect_match_columns(sequences)

In [ ]:
# Para usar en los estados de inserción
def calculate_background_freq(sequences):
    freq = defaultdict(int)
    total = 0
    for seq in sequences:
        for res in seq:
            if res != "-":
                freq[res] += 1
                total += 1
    return {res: count / total for res, count in freq.items()}

background_freq = calculate_background_freq(sequences)

In [ ]:
def build_profile_hmm(sequences, match_cols):
    emissions = defaultdict(lambda: defaultdict(int))
    transitions = defaultdict(lambda: defaultdict(int))

    for seq in sequences:
        prev_state = "START"
        match_idx = 1

        for i, res in enumerate(seq):
            if match_cols[i]:  # columna tipo MATCH o DELETE
                if res == "-":
                    state = f"D{match_idx}"
                else:
                    state = f"M{match_idx}"
                    emissions[state][res] += 1
                transitions[prev_state][state] += 1
                prev_state = state
                match_idx += 1
            else:  # columna tipo INSERT
                if res != "-":
                    state = f"I{match_idx - 1}"
                    emissions[state][res] += 1
                    transitions[prev_state][state] += 1
                    prev_state = state
        transitions[prev_state]["END"] += 1

    return emissions, transitions


In [ ]:
 # Import the math module

def normalize_log_probs(counts, pseudocount=1e-6):
    probs = {}
    for state in counts:
        total = sum(counts[state].values()) + pseudocount * len(counts[state])
        probs[state] = {
            k: math.log((v + pseudocount) / total) # Now math.log will be recognized
            for k, v in counts[state].items()
        }
    return probs

In [ ]:
emissions, transitions = build_profile_hmm(sequences, match_cols)
log_emissions = normalize_log_probs(emissions)
log_transitions = normalize_log_probs(transitions)

In [ ]:
from pprint import pprint

print("Log-transiciones desde M1:")
pprint(log_transitions.get("M1", {}))

print("\nLog-emisiones desde M1:")
pprint(log_emissions.get("M1", {}))

print("\nFrecuencia global p(a) para inserciones:")
pprint(background_freq)


Log-transiciones desde M1:
{'D2': -6.145614895030534, 'M2': -0.0021451570591398735}

Log-emisiones desde M1:
{'A': -5.857932912340633,
 'C': -5.6347894110264125,
 'F': -5.857932912340633,
 'K': -6.145614901459106,
 'L': -4.153185024647633,
 'M': -0.03636765128727696,
 'P': -7.244226523460993,
 'S': -5.6347894110264125,
 'T': -6.5510798429006725,
 'V': -6.145614901459106,
 'X': -7.244226523460993}

Frecuencia global p(a) para inserciones:
{'A': 0.07005214237134437,
 'B': 1.3335644845106485e-05,
 'C': 0.0995239174790297,
 'D': 0.05342259324949658,
 'E': 0.03831997546241348,
 'F': 0.036239614866576876,
 'G': 0.06513128942350008,
 'H': 0.020170162828223558,
 'I': 0.035312787549841976,
 'K': 0.046654753490605036,
 'L': 0.09956392441356501,
 'M': 0.03461266619547388,
 'N': 0.037326469921453054,
 'O': 0.0001266886260285116,
 'P': 0.04926853988024591,
 'Q': 0.030225239041433847,
 'R': 0.06716497526237881,
 'S': 0.06654486777708137,
 'T': 0.056849853974688946,
 'V': 0.058243428861002575,
 'W': 

In [ ]:
# Algoritmo de Viterbi en log-space
def viterbi(sequence, log_transitions, log_emissions, match_columns):
    states = sorted(log_emissions.keys(), key=lambda x: (x[0], int(x[1:])))
    seq_len = len(sequence)
    n_states = len(states)

    # Inicialización
    V = np.full((n_states, seq_len + 1), -np.inf)
    backpointer = np.zeros((n_states, seq_len + 1), dtype=int)

    state_index = {state: idx for idx, state in enumerate(states)}

    # Estados que pueden seguir a START
    for state in states:
        if "START" in log_transitions and state in log_transitions["START"]:
            emis = log_emissions[state].get(sequence[0], -np.inf)
            V[state_index[state], 1] = log_transitions["START"][state] + emis

    # Recursión
    for t in range(2, seq_len + 1):
        for curr_state in states:
            emis = log_emissions[curr_state].get(sequence[t - 1], -np.inf)
            max_prob = -np.inf
            max_state = 0
            for prev_state in states:
                if prev_state in log_transitions and curr_state in log_transitions[prev_state]:
                    prob = V[state_index[prev_state], t - 1] + log_transitions[prev_state][curr_state] + emis
                    if prob > max_prob:
                        max_prob = prob
                        max_state = state_index[prev_state]
            V[state_index[curr_state], t] = max_prob
            backpointer[state_index[curr_state], t] = max_state

    # Terminación
    best_path_prob = -np.inf
    best_last_state = 0
    for state in states:
        if state in log_transitions and "END" in log_transitions[state]:
            prob = V[state_index[state], seq_len] + log_transitions[state]["END"]
            if prob > best_path_prob:
                best_path_prob = prob
                best_last_state = state_index[state]

    # Backtracking
    best_path = [best_last_state]
    for t in range(seq_len, 1, -1):
        best_state = backpointer[best_path[0], t]
        best_path.insert(0, best_state)

    best_state_path = [states[i] for i in best_path]

    return best_state_path, best_path_prob

# Prueba con una secuencia ejemplo
test_sequence = "MCCGKCLRKCG"
sequence_clean = [aa for aa in test_sequence if aa != '-']
best_path, score = viterbi(sequence_clean, log_transiciones, log_emisiones, match_columns)

best_path, score

(['I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0', 'I0'], -inf)

In [ ]:
from collections import defaultdict

def build_hmm_from_alignment(seqs, match_columns):
    emissions = defaultdict(lambda: defaultdict(int))
    transitions = defaultdict(lambda: defaultdict(int))

    for seq in seqs:
        prev_state = "START"
        match_idx = 1

        for i, res in enumerate(seq):
            if match_columns[i]:  # Match o Delete
                if res == "-":
                    state = f"D{match_idx}"
                else:
                    state = f"M{match_idx}"
                    emissions[state][res] += 1
                transitions[prev_state][state] += 1
                prev_state = state
                match_idx += 1
            else:  # Inserción
                if res != "-":
                    state = f"I{match_idx - 1}"
                    emissions[state][res] += 1
                    transitions[prev_state][state] += 1
                    prev_state = state
        transitions[prev_state]["END"] += 1

    return transitions, emissions

In [ ]:
def normalize_counts(counts):
    probs = {}
    for state in counts:
        total = sum(counts[state].values())
        probs[state] = {k: v / total for k, v in counts[state].items()}
    return probs

In [ ]:
match_columns = identify_match_columns(aligned_seqs, threshold=0.5)
transitions, emissions = build_hmm_from_alignment(aligned_seqs, match_columns)

trans_probs = normalize_counts(transitions)
emiss_probs = normalize_counts(emissions)

In [ ]:
from pprint import pprint

print("\nTransiciones desde M1:")
pprint(trans_probs.get("M1", {}))

print("\nEmisiones desde M1:")
pprint(emiss_probs.get("M1", {}))



Transiciones desde M1:
{'D2': 0.002142857142857143, 'M2': 0.9978571428571429}

Emisiones desde M1:
{'A': 0.002857142857142857,
 'C': 0.0035714285714285713,
 'F': 0.002857142857142857,
 'K': 0.002142857142857143,
 'L': 0.015714285714285715,
 'M': 0.9642857142857143,
 'P': 0.0007142857142857143,
 'S': 0.0035714285714285713,
 'T': 0.0014285714285714286,
 'V': 0.002142857142857143,
 'X': 0.0007142857142857143}


In [ ]:
!pip install hmmlearn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from hmmlearn import hmm

file_fasta = "/content/drive/MyDrive/Proyecto/aligned_cluster_sequences.fasta"

#### Construcción del perfil HMM

* Definir los estados: Match, insert and delete.
* Calcular las probabilidades de emisión
* Calcular las probabilidades de transición
* Implementación del algoritmo Viterbi
- Evaluación de nuevas secuencias a partir del modelo HMM


In [ ]:
import numpy as np
from collections import defaultdict

def build_profile_hmm(sequences, match_threshold=0.5):
    num_seqs = len(sequences)
    align_len = len(sequences[0])

    # Identificar columnas de Match
    match_cols = []
    for col in range(align_len):
        gap_count = sum(1 for seq in sequences if seq[col] == '-')
        is_match = (gap_count / num_seqs) < match_threshold
        match_cols.append(is_match)

    # Inicializar matrices de emisiones y transiciones
    emissions = defaultdict(lambda: defaultdict(int))
    transitions = defaultdict(lambda: defaultdict(int))

    # Contar emisiones y transiciones
    for seq in sequences:
        prev_state = "START"
        match_idx = 1

        for i, res in enumerate(seq):
            if match_cols[i]:  # Columna de Match o Delete
                if res == "-":
                    state = f"D{match_idx}"
                else:
                    state = f"M{match_idx}"
                    emissions[state][res] += 1
                transitions[prev_state][state] += 1
                prev_state = state
                match_idx += 1
            else:  # Columna de Insert
                if res != "-":
                    state = f"I{match_idx - 1}"
                    emissions[state][res] += 1
                    transitions[prev_state][state] += 1
                    prev_state = state
        transitions[prev_state]["END"] += 1

    # Normalizar a probabilidades
    def normalize_counts(counts):
        probs = {}
        for state in counts:
            total = sum(counts[state].values())
            probs[state] = {k: v / total for k, v in counts[state].items()}
        return probs

    emission_probs = normalize_counts(emissions)
    transition_probs = normalize_counts(transitions)

    return emission_probs, transition_probs


Algoritmo de Viterbi

In [ ]:
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import AlignIO
from Bio import SeqIO
import os
import subprocess
from collections import defaultdict
import pprint

print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir('.'))
def extract_cluster_sequences(cluster_filename,fasta_filename,output_filename):
    with open(cluster_filename,'r',encoding='utf-8') as cluster_file:
        cluster_ids = []
        for line in cluster_file:
            if line.startswith('>'):
                continue
            if(line.__contains__("*")):
                seq_id = line.split('>')[1].split('...')[0]
            cluster_ids.append(seq_id)
    with open(fasta_filename, 'r', encoding='utf-8') as fasta_file:
        sequences = SeqIO.parse(fasta_file, 'fasta')
        filtered_seqs = [seq for seq in sequences if seq.id in cluster_ids]
    with open(output_filename, 'w', encoding='utf-8') as output:
        SeqIO.write(filtered_seqs, output, 'fasta')
# 2. Identificar las columnas "match" basadas en el umbral de no-gap (>=50%)

def read_sequences(align_filename):
    alignment = []
    with open(align_filename,'r',encoding='utf-8') as align_file:
        current_line = ""
        for line in align_file:
            if line.startswith('>'):
                alignment.append(current_line)
                current_line =''
                continue
            else:
                current_line+=line
                current_line=current_line.replace('\n',"")
    return alignment

# extract_cluster_sequences(
#     cluster_filename='/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/conoserver_250206_protein_cdhit.fa.clstr',
#     fasta_filename='/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/conoserver_250206_protein.fa',
#     output_filename='/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/cluster_sequences.fasta'
# )

def identify_match_columns(alignment, threshold=0.5):
    nseq = len(alignment)
    ncol = len(alignment[1])
    match_cols =[]
    for j in range(ncol):
        count_non_gap = sum(1 for seq in alignment if seq[j] != '-')
        if count_non_gap / nseq >= threshold:
            match_cols.append(j)
    return match_cols

# Construir la secuencia de estados para cada secuencia
#     - Para cada columna match: si no es gap -> estado M; Si lo es -> estado D.
#     - Para cada columna que no es match: si hay residuo -> estado I asociado al ultimo match.
#     - Se incluyen estados de inicio 'S' y fin 'E'

def get_state_path(seq,match_cols):
    state_path =['S']
    match_index = 0
    last_match_index = None
    ncol = len(seq)
    for j in range(ncol):
        if j in match_cols:
            match_index +=1
            if seq[j] =='-':
                state_path.append(f'D{match_index}')
            else:
                state_path.append(f'M{match_index}')
            last_match_index = match_index
        else:
            # Columna de insercion
            if seq[j] !='-':
                # El estado de incersion se asocia al ultimo match (o 0 si no hay match previo)
                ins_state = f'I{last_match_index if last_match_index is not None else 0}'
                state_path.append(ins_state)
    state_path.append('E')
    return state_path

# Calculo de las transiciones entre estados
def calculate_transitions(state_paths):
    transition_counts = defaultdict(int)
    for path in state_paths:
        for i in range(len(path)-1):
            transition = (path[i],path[i+1])
            transition_counts[transition]+=1
    return transition_counts

# Calculo de la matriz de emisiones
def get_emissions(seq, match_cols):
    emissions = []
    match_index = 0
    last_match_index = None
    ncol = len(seq)
    for j in range(ncol):
        if j in match_cols:
            match_index +=1
            if seq[j] != '-':
                state = f'M{match_index}'
                emissions.append((state,seq[j]))
            # Si es gap, se considera delecion (no emite)
            last_match_index = match_index
        else:
            # Columna de insercion: si hay residuo, emite
            if seq[j] !='-':
                ins_state=f'I{last_match_index if last_match_index is not None else 0}'
                emissions.append((ins_state,seq[j]))
    return emissions


# input_fasta = './BioInf_ProyectoFinal/cluster_sequences.fasta'
# output_fasta = './BioInf_ProyectoFinal/aligned_cluster_sequences_01.fasta'

# cmd = ['mafft.bat', '--anysymbol','--localpair', input_fasta]
# result = subprocess.run(cmd,capture_output=True,text=True,encoding='utf-8')

# if result.returncode != 0:
#     print("MAFFT Error:", result.stderr)
# else:
#     with open(output_fasta, 'w', encoding='utf-8') as aligned_output:
#         aligned_output.write(result.stdout)

# alignment = AlignIO.read(output_fasta, 'fasta')
# print(alignment)
alignment = read_sequences('/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/Cluster95_alineamiento.fasta')
alignment.pop(0)

match_cols = identify_match_columns(alignment, threshold=0.5)
# Generar la trayectoria de estados para cada secuencia
state_paths = [get_state_path(seq,match_cols) for seq in alignment]
for i, path in enumerate(state_paths):
    print(f'Secuencia {i+1} - trayectoria de estados: {path}')

# Generar las transiciones entre estados
transition_counts = calculate_transitions(state_paths)
for trans,count in transition_counts.items():
    print(f"{trans}: {count}")

# Generar matriz de emisiones
emission_counts = defaultdict(lambda:defaultdict(int))
for seq in alignment:
    for state, symbol in get_emissions(seq,match_cols):
        emission_counts[state][symbol]+=1

print("\nContabilización de Emisiones (por estado):")
for state, counts in emission_counts.items():
    print(f"{state}: {dict(counts)}")

# Calcular las probabilidades de emision y transicion (usando pseudoconteos)
alpha= 1
protein_alphabet = list("ARNDCEQGHILKMFPSTWYVUOBZJ")
emission_probabilities = {}
for state,counts in emission_counts.items():
    total = sum(counts.values())+ alpha * len(protein_alphabet)
    emission_probabilities[state]={}
    for letter in protein_alphabet:
        emission_probabilities[state][letter] = (counts.get(letter,0)+alpha)/total

print("\nProbabilidades de Emisión (con pseudocontadores):")
pprint.pprint(emission_probabilities)

alpha_trans = 1  # Pseudocontador para transiciones
from_state_transitions = defaultdict(lambda:defaultdict(int))
for(from_state,to_state), count in transition_counts.items():
    from_state_transitions[from_state][to_state] =count

transition_probabilities = {}
for from_state,transitions in from_state_transitions.items():
    # Numero de transiciones posibles observadas desde 'from_state'
    total = sum(transitions.values())+alpha_trans *len(transitions)
    transition_probabilities[from_state]={}
    for to_state,count in transitions.items():
        transition_probabilities[from_state][to_state] = (count+alpha_trans)/total

print("\nProbabilidades de Transición (con pseudocontadores):")
pprint.pprint(dict(transition_probabilities))

Current working directory: /content
Files in current directory: ['.config', 'drive', 'sample_data']
Secuencia 1 - trayectoria de estados: ['S', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'I18', 'M19', 'D20', 'M21', 'M22', 'M23', 'M24', 'M25', 'D26', 'M27', 'M28', 'I28', 'I28', 'I28', 'M29', 'M30', 'I30', 'M31', 'I31', 'I31', 'I31', 'M32', 'M33', 'I33', 'M34', 'D35', 'D36', 'M37', 'D38', 'I38', 'M39', 'I39', 'I39', 'D40', 'D41', 'M42', 'I42', 'I42', 'I42', 'I42', 'I42', 'I42', 'M43', 'M44', 'D45', 'I45', 'E']
Secuencia 2 - trayectoria de estados: ['S', 'I0', 'I0', 'I0', 'I0', 'I0', 'M1', 'D2', 'M3', 'M4', 'M5', 'M6', 'D7', 'D8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'I14', 'I14', 'I14', 'I14', 'M15', 'M16', 'M17', 'I17', 'I17', 'I17', 'M18', 'I18', 'M19', 'D20', 'M21', 'M22', 'M23', 'M24', 'M25', 'D26', 'M27', 'M28', 'I28', 'I28', 'I28', 'M29', 'M30', 'I30', 'M31', 'I31', 'I31', 'I31', 'M32', 'M33', 'I33', 'M34

In [ ]:
import math
class HMM:
    def __init__(self,states,start_state,end_state,trans_probs,emiss_probs):
        self.states= states
        self.start_state = start_state
        self.end_state = end_state
        self.trans_probs = trans_probs
        self.emiss_probs = emiss_probs
    def viterbi(self,observations):
        """
        Implementacion del algoritmo de Viterbi usando log-probabilidades para mayor estabilidad.
        observations: lista o cadena de observaciones (simbolos de aminoacidos)
        Retorna la secuencia de estados mas probable y el log de la probabilidad de esa trayectoria
        """

        T= len(observations)
        # dp[t] sera un diccionario que contiene la log-probabilidad maxima para cada estado al tiempo t
        # bp[t] contendra el backpointer para reconstruir la trayectoria
        dp =[{} for _ in range(T+1)]
        bp =[{} for _ in range(T+1)]

        # Inicializacion: desde el estado S
        dp[0][self.start_state] = 0
        # Para t = 1 a T, iterar sobre cada observacion
        for t in range(1,T+1):
            o = observations[t-1]
            for curr in self.states:
                # Solo consideramos estados emisores: ignorar S y E
                if curr in [self.start_state, self.end_state]:
                    continue
                max_log_prob = -math.inf
                best_prev = None
                # Revisamos todas las posibles transiciones provenientes de los estados con los que terminamos en el paso anterior
                for prev, log_prob_prev in dp[t-1].items():
                    # Verificamos que exista una transicion de prev a curr
                    if prev in self.trans_probs and curr in self.trans_probs[prev]:
                        # Obtenemos el log de la probabilidad de transición
                        log_trans= math.log(self.trans_probs[prev][curr])
                        # Obtenemos el log de la probabilidad de emisión en el estado curr para la observación actual
                        # Si la emision no esta definida, asignamos un valor pequeño para evitar log(0)
                        emiss_p =self.emiss_probs.get(curr,{}).get(o,1e-10)
                        log_emit = math.log(emiss_p)
                        candidate = log_prob_prev+log_trans+log_emit
                        if candidate > max_log_prob:
                            max_log_prob = candidate
                            best_prev = prev
                # Solo agregamos el estado si se encontro al menos un camino
                if best_prev is not None:
                    dp[t][curr] = max_log_prob
                    bp[t][curr] = best_prev

        # Paso de terminacion: Transitar desde alguno de los estados emisores al estado final
        max_log_prob = -math.inf
        best_last_state = None
        # Ensure 'E' is in transition_probabilities for each state in dp[T]
        for state, log_prob in dp[T].items():
            if state in self.trans_probs and self.end_state in self.trans_probs[state]:
                log_trans = math.log(self.trans_probs[state][self.end_state])
                candidate = log_prob + log_trans
                if candidate > max_log_prob:
                    max_log_prob = candidate
                    best_last_state = state
            # If 'E' isn't a direct transition, add a small probability to allow the path to end
            elif state in self.trans_probs and self.end_state not in self.trans_probs[state] and state != 'E':
                # Adding a pseudo transition to end state to handle cases where there's no defined path
                log_trans = math.log(1e-10)
                candidate = log_prob + log_trans
                if candidate > max_log_prob:
                    max_log_prob = candidate
                    best_last_state = state

        # If valid path to end state is not found, raise an exception
        if best_last_state is None:
            raise ValueError("No se encontró una trayectoria válida hacia el estado final")

        # Backtracking to construct the path
        path = [self.end_state]
        curr_state = best_last_state
        for t in range(T, 0, -1):
            path.append(curr_state)
            curr_state = bp[t][curr_state]
        path.append(self.start_state)
        path = list(reversed(path))
        return path, max_log_prob

In [ ]:
test_sequence = list("MCCGKCLRKCG")
hmm = HMM(
    states=list(emission_probabilities.keys()),
    start_state="S",
    end_state="E",
    trans_probs=transition_probabilities,
    emiss_probs=emission_probabilities
)
ruta, score = hmm.viterbi(test_sequence)
print("Ruta más probable:", ruta)
print("Log-score:", math.exp(score))


NameError: name 'HMM' is not defined

In [ ]:
!pip install hmmlearn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
amino_acids = "ARNDCQEGHILKMFPSTWYV"
aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}

In [ ]:
def normalize(arr):
    return arr / np.sum(arr)

def forward(obs, A, B, pi):
    T, N = len(obs), A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = pi * B[:, obs[0]]
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = np.sum(alpha[t-1] * A[:, j]) * B[j, obs[t]]
    return alpha

def backward(obs, A, B):
    T, N = len(obs), A.shape[0]
    beta = np.zeros((T, N))
    beta[T-1] = np.ones(N)
    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t, i] = np.sum(A[i] * B[:, obs[t+1]] * beta[t+1])
    return beta

def baum_welch_multiple(sequences, N, M, max_iters=100, tol=1e-6):
    A = normalize(np.random.rand(N, N))
    B = normalize(np.random.rand(N, M))
    pi = normalize(np.random.rand(N))

    for _ in range(max_iters):
        A_num = np.zeros((N, N))
        A_den = np.zeros(N)
        B_num = np.zeros((N, M))
        B_den = np.zeros(N)
        pi_accum = np.zeros(N)

        total_log_prob = 0

        for seq in sequences:
            obs = [aa_to_index[aa] for aa in seq]
            T = len(obs)

            alpha = forward(obs, A, B, pi)
            beta = backward(obs, A, B)

            log_prob = np.log(np.sum(alpha[-1]))
            total_log_prob += log_prob

            gamma = (alpha * beta) / np.sum(alpha * beta, axis=1, keepdims=True)
            xi = np.zeros((T-1, N, N))

            for t in range(T-1):
                denom = np.sum([
                    alpha[t, i] * A[i, j] * B[j, obs[t+1]] * beta[t+1, j]
                    for i in range(N) for j in range(N)
                ])
                for i in range(N):
                    for j in range(N):
                        xi[t, i, j] = alpha[t, i] * A[i, j] * B[j, obs[t+1]] * beta[t+1, j] / denom

            pi_accum += gamma[0]
            A_num += np.sum(xi, axis=0)
            A_den += np.sum(gamma[:-1], axis=0)

            for t in range(T):
                B_num[:, obs[t]] += gamma[t]
                B_den += gamma[t]

        # Actualizar A, B, pi
        A = A_num / A_den[:, None]
        B = B_num / B_den[:, None]
        pi = normalize(pi_accum)

        # Convergencia (opcional)
        # print(f"Log-likelihood: {total_log_prob:.4f}")

    return A, B, pi

In [ ]:

sequences = AlignIO.read("/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/Cluster95_alineamiento.fasta", "fasta")

N = 3  # Número de estados ocultos
M = 20  # 20 aminoácidos

A, B, pi = baum_welch_multiple(sequences, N, M)

print("Matriz de transición A:")
print(A)
print("Matriz de emisión B:")
print(B)
print("Distribución inicial π:")
print(pi)

KeyError: '-'

# Nuevo comienzo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import gzip
from typing import List, Tuple
import os

def parse_fasta(file_path: str) -> List[str]:
    """
    Parsea archivos FASTA (normales o comprimidos)

    Args:
        file_path: Ruta al archivo FASTA

    Returns:
        Lista de secuencias
    """
    sequences = []
    current_seq = []

    # Verificar si el archivo existe
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"El archivo {file_path} no existe")

    # Decidir si abrir como gz o normal
    opener = gzip.open if file_path.endswith('.gz') else open

    try:
        with opener(file_path, 'rt') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if current_seq:
                        sequences.append(''.join(current_seq))
                        current_seq = []
                else:
                    current_seq.append(line)

            # No olvidar la última secuencia
            if current_seq:
                sequences.append(''.join(current_seq))
    except Exception as e:
        raise Exception(f"Error al leer el archivo FASTA: {e}")

    # Verificación básica
    if not sequences:
        raise ValueError("No se encontraron secuencias en el archivo FASTA")

    print(f"Se leyeron {len(sequences)} secuencias del archivo FASTA")
    return sequences

def filter_alignment_columns(sequences: List[str], min_occupancy: float = 0.1) -> Tuple[List[str], List[int]]:
    """
    Filtra las columnas del alineamiento que tienen una ocupancia menor al umbral

    Args:
        sequences: Lista de secuencias alineadas
        min_occupancy: Fracción mínima de posiciones no-gap requerida (0-1)

    Returns:
        Tupla con (secuencias_filtradas, columnas_conservadas)
    """
    if not sequences:
        return [], []

    alignment_length = len(sequences[0])
    num_sequences = len(sequences)

    # Verificar que todas las secuencias tengan la misma longitud
    for i, seq in enumerate(sequences):
        if len(seq) != alignment_length:
            raise ValueError(f"La secuencia {i} tiene longitud {len(seq)}, pero debería ser {alignment_length}")

    print(f"Analizando alineamiento: {num_sequences} secuencias de longitud {alignment_length}")

    # Calcular ocupancia por columna
    occupancy = []
    for col in range(alignment_length):
        non_gaps = sum(1 for seq in sequences if seq[col] != '-')
        occupancy.append(non_gaps / num_sequences)

    # Identificar columnas a mantener
    columns_to_keep = [col for col, occ in enumerate(occupancy) if occ >= min_occupancy]

    # Filtrar secuencias
    filtered_sequences = []
    for seq in sequences:
        filtered_seq = ''.join(seq[col] for col in columns_to_keep)
        filtered_sequences.append(filtered_seq)

    print(f"Filtrado: de {alignment_length} columnas originales a {len(columns_to_keep)} columnas ({len(columns_to_keep)/alignment_length*100:.1f}%)")

    # Verificar la longitud promedio sin gaps después del filtrado
    avg_non_gaps = sum(sum(1 for char in seq if char != '-') for seq in filtered_sequences) / num_sequences
    print(f"Promedio de caracteres no-gap por secuencia: {avg_non_gaps:.1f} de {len(columns_to_keep)} ({avg_non_gaps/len(columns_to_keep)*100:.1f}%)")

    return filtered_sequences, columns_to_keep

# HMM

In [6]:
from collections import defaultdict
from typing import Dict, Tuple, List, Any
import numpy as np

def build_hmm(sequences: List[str], pseudocount: float = 5.0, max_model_size: int = 1343) -> Tuple:
    """
    Construye las matrices de transición y emisión del HMM con estados Match, Insert y Delete.

    Args:
        sequences: Lista de secuencias alineadas
        pseudocount: Valor de pseudoconteo para evitar probabilidades cero

    Returns:
        Tupla con (trans_probs, emit_probs, amino_acids, states)
    """
    # Validación de entrada
     # Validación de entrada
    if not sequences:
        raise ValueError("No se proporcionaron secuencias")

    alignment_length = len(sequences[0])

    # Si el alineamiento es demasiado grande, filtrar posiciones con alta ocupancia
    if alignment_length > max_model_size:
        print(f"⚠️ Alineamiento muy grande ({alignment_length}), filtrando para mantener {max_model_size} posiciones")
        # Calcular ocupancia por columna
        column_occupancy = []
        for col in range(alignment_length):
            non_gaps = sum(1 for seq in sequences if seq[col] != '-')
            column_occupancy.append(non_gaps / len(sequences))

        # Seleccionar las columnas con mayor ocupancia
        cols_to_keep = sorted(range(alignment_length), key=lambda i: column_occupancy[i], reverse=True)[:max_model_size]
        cols_to_keep.sort()  # Reordenar para mantener el orden original

        # Filtrar secuencias
        filtered_sequences = []
        for seq in sequences:
            filtered_seq = ''.join(seq[col] for col in cols_to_keep)
            filtered_sequences.append(filtered_seq)

        sequences = filtered_sequences
        alignment_length = len(sequences[0])
        print(f"Alineamiento reducido a {alignment_length} posiciones")

    # Estados del HMM
    states = ['S', 'I0'] + \
             [f'M{i}' for i in range(1, alignment_length+1)] + \
             [f'I{i}' for i in range(1, alignment_length+1)] + \
             [f'D{i}' for i in range(1, alignment_length+1)] + \
             ['E']

    # Recolectar todos los aminoácidos únicos (excluyendo gaps explícitamente)
    amino_acids = set()
    for seq in sequences:
        amino_acids.update(char for char in seq if char != '-')

    # Verificar aminoácidos válidos
    standard_aas = set("ACDEFGHIKLMNPQRSTVWY")
    non_standard = amino_acids - standard_aas
    if non_standard:
        print(f"⚠️ Advertencia: Se encontraron aminoácidos no estándar: {non_standard}")
        print("   Estos serán tratados como caracteres normales")

    # Convertir a lista ordenada para uso posterior
    amino_acids = sorted(amino_acids)

    # Inicializar contadores con pseudocounts
    trans_counts = defaultdict(lambda: defaultdict(lambda: pseudocount))
    match_emit_counts = defaultdict(lambda: defaultdict(lambda: pseudocount))
    ins_emit_counts = defaultdict(lambda: defaultdict(lambda: pseudocount))

    # Contar transiciones y emisiones
    print("Contando transiciones y emisiones...")

    try:
        for i, seq in enumerate(sequences):
            if i % 100 == 0:  # Mostrar progreso cada 100 secuencias
                print(f"Procesando secuencia {i}/{len(sequences)} ({i/len(sequences)*100:.1f}%)")

            # Verificar caracteres válidos
            invalid_chars = set(seq) - set('ACDEFGHIKLMNPQRSTVWY-')
            if invalid_chars:
                print(f"⚠️ Secuencia {i} tiene caracteres no válidos: {invalid_chars}")
                # Limpiar la secuencia (reemplazar caracteres no válidos con '-')
                seq = ''.join(char if char in "ACDEFGHIKLMNPQRSTVWY-" else '-' for char in seq)

            prev_state = 'S'

            for pos, aa in enumerate(seq):
                pos_index = pos + 1  # Ajuste para índices basados en 1

                if aa == '-':
                    # Gap representa un estado Delete en la posición
                    current_state = f'D{pos_index}'
                else:
                    # Un aminoácido en la posición corresponde a un Match
                    current_state = f'M{pos_index}'
                    match_emit_counts[current_state][aa] += 1

                # Registrar transición
                trans_counts[prev_state][current_state] += 1
                prev_state = current_state

            # Transición al estado final
            trans_counts[prev_state]['E'] += 1

    except Exception as e:
        print(f"Error en secuencia {i}: {e}")
        raise

    # Inicializar la topología completa del modelo y calcular probabilidades
    print("Calculando matrices de probabilidad...")

    # Definir la estructura del HMM: qué transiciones son posibles
    valid_transitions = {
        'S': ['M1', 'I0', 'D1'],
        'I0': ['M1', 'I0', 'D1']
    }

    # Para cada posición, definir las transiciones válidas
    for i in range(1, alignment_length):
        valid_transitions[f'M{i}'] = [f'M{i+1}', f'I{i}', f'D{i+1}']
        valid_transitions[f'I{i}'] = [f'M{i+1}', f'I{i}', f'D{i+1}']
        valid_transitions[f'D{i}'] = [f'M{i+1}', f'I{i}', f'D{i+1}']

    # Últimas posiciones: solo pueden transitar al estado final 'E'
    valid_transitions[f'M{alignment_length}'] = ['E']
    valid_transitions[f'I{alignment_length}'] = ['E']
    valid_transitions[f'D{alignment_length}'] = ['E']

    # Calcular probabilidades de transición
    trans_probs = {}
    for src in states:
        trans_probs[src] = {}
        # Si no hay transiciones definidas, usar las válidas para el tipo de estado
        if src not in valid_transitions:
            continue

        valid_dests = valid_transitions[src]

        # Calcular total de transiciones para normalización
        total = sum(trans_counts[src][dst] for dst in valid_dests)

        if total <= pseudocount * len(valid_dests):
            # Si no hay suficientes transiciones, distribuir uniformemente
            for dst in valid_dests:
                trans_probs[src][dst] = 1.0 / len(valid_dests)
        else:
            for dst in valid_dests:
                trans_probs[src][dst] = trans_counts[src][dst] / total

    # Probabilidades de emisión
    emit_probs = {}

    # Para estados de Match
    for i in range(1, alignment_length+1):
        state = f'M{i}'
        emit_probs[state] = {}

        total = sum(match_emit_counts[state][aa] for aa in amino_acids)

        if total <= pseudocount * len(amino_acids):
            # Si no hay emisiones, distribuir uniformemente
            for aa in amino_acids:
                emit_probs[state][aa] = 1.0 / len(amino_acids)
        else:
            for aa in amino_acids:
                emit_probs[state][aa] = match_emit_counts[state][aa] / total

    # Para estados de Inserción
    for i in range(0, alignment_length+1):
        state = f'I{i}'
        emit_probs[state] = {}

        total = sum(ins_emit_counts[state][aa] for aa in amino_acids)

        if total <= pseudocount * len(amino_acids):
            # Si no hay emisiones, distribuir uniformemente
            for aa in amino_acids:
                emit_probs[state][aa] = 1.0 / len(amino_acids)
        else:
            for aa in amino_acids:
                emit_probs[state][aa] = ins_emit_counts[state][aa] / total

    # Verificar NaN o inf en las matrices
    for src in trans_probs:
        for dst in trans_probs[src]:
            if not np.isfinite(trans_probs[src][dst]):
                print(f"⚠️ Advertencia: Valor no finito en trans_probs[{src}][{dst}] = {trans_probs[src][dst]}")
                trans_probs[src][dst] = 0.01  # Valor de respaldo razonable

    for state in emit_probs:
        for aa in emit_probs[state]:
            if not np.isfinite(emit_probs[state][aa]):
                print(f"⚠️ Advertencia: Valor no finito en emit_probs[{state}][{aa}] = {emit_probs[state][aa]}")
                emit_probs[state][aa] = 0.01  # Valor de respaldo razonable

    # Imprimir algunas estadísticas para verificar
    print(f"Total de estados: {len(states)}")
    print(f"Estados Match: {len([s for s in states if s.startswith('M')])}")
    print(f"Estados Insert: {len([s for s in states if s.startswith('I')])}")
    print(f"Estados Delete: {len([s for s in states if s.startswith('D')])}")
    print(f"Total de aminoácidos: {len(amino_acids)}")

    return trans_probs, emit_probs, amino_acids, states

def filter_high_gap_columns(sequences, max_gap_ratio=0.8):
    """Elimina columnas con demasiados gaps"""
    if not sequences:
        return []

    alignment_length = len(sequences[0])
    num_seqs = len(sequences)

    # Identificar columnas a mantener
    columns_to_keep = []
    for pos in range(alignment_length):
        gap_count = sum(1 for seq in sequences if seq[pos] == '-')
        if gap_count / num_seqs < max_gap_ratio:
            columns_to_keep.append(pos)

    # Filtrar secuencias
    filtered_seqs = []
    for seq in sequences:
        new_seq = ''.join(seq[i] for i in columns_to_keep)
        filtered_seqs.append(new_seq)

    return filtered_seqs
def build_hmm_filtered(sequences: List[str], min_occupancy: float = 0.1, pseudocount: float = 1.0) -> Tuple:
    """
    Construye un HMM después de filtrar columnas con demasiados gaps

    Args:
        sequences: Lista de secuencias alineadas
        min_occupancy: Fracción mínima de posiciones no-gap requerida (0-1)
        pseudocount: Valor de pseudoconteo para evitar probabilidades cero

    Returns:
        Tupla con (trans_probs, emit_probs, amino_acids, states)
    """
    from .fasta_parser import filter_alignment_columns

    # Filtrar columnas con muchos gaps
    filtered_sequences, _ = filter_alignment_columns(sequences, min_occupancy)

    # Construir HMM con las secuencias filtradas
    return build_hmm(filtered_sequences, pseudocount)

# Baum Welch

In [7]:
import numpy as np
from typing import Dict, List, Tuple, Any
from collections import defaultdict

def forward_algorithm_fixed(sequence: str,
                          trans_probs: Dict[str, Dict[str, float]],
                          emit_probs: Dict[str, Dict[str, float]],
                          states: List[str],
                          amino_acids: List[str]) -> float:
    """Algoritmo Forward mejorado para matrices de grandes dimensiones"""
    # Limpiar secuencia de caracteres no estándar
    cleaned_sequence = ''.join(ch if ch in amino_acids else amino_acids[0] for ch in sequence)
    sequence = cleaned_sequence

    # Convertir a logaritmos para evitar underflow
    log_trans = {s: {d: np.log(p) if p > 0 else -np.inf
                    for d, p in dests.items()}
                for s, dests in trans_probs.items()}

    log_emit = {s: {a: np.log(p) if p > 0 else -np.inf
                   for a, p in aa_probs.items()}
               for s, aa_probs in emit_probs.items()}

    # Inicializar matriz forward
    T = len(sequence)
    forward = [{state: -np.inf for state in states} for _ in range(T + 1)]

    # Estado inicial
    forward[0]['S'] = 0.0  # log(1)

    # Procesar estados Delete iniciales (antes de observar símbolos)
    for state in states:
        if state.startswith('D'):
            if 'S' in trans_probs and state in trans_probs['S']:
                forward[0][state] = log_trans['S'][state]

            # Propagación a través de más estados Delete
            for next_d in states:
                if next_d.startswith('D') and state in trans_probs and next_d in trans_probs[state]:
                    if forward[0][state] > -np.inf:
                        new_prob = forward[0][state] + log_trans[state][next_d]
                        forward[0][next_d] = np.logaddexp(forward[0][next_d], new_prob)

    # Procesar la secuencia símbolo por símbolo
    for t in range(T):
        symbol = sequence[t]

        # Para cada estado en esta posición
        for curr_state in states:
            if forward[t][curr_state] <= -np.inf:
                continue  # Omitir estados imposibles

            # Procesar estados que emiten símbolos (Match e Insert)
            for next_state in states:
                if curr_state in trans_probs and next_state in trans_probs[curr_state]:
                    # Estados que emiten
                    if (next_state.startswith('M') or next_state.startswith('I')):
                        if symbol in log_emit.get(next_state, {}):
                            emit_prob = log_emit[next_state][symbol]
                            new_prob = forward[t][curr_state] + log_trans[curr_state][next_state] + emit_prob
                            forward[t+1][next_state] = np.logaddexp(forward[t+1][next_state], new_prob)

                    # Estados que no emiten (Delete)
                    elif next_state.startswith('D'):
                        new_prob = forward[t][curr_state] + log_trans[curr_state][next_state]
                        forward[t][next_state] = np.logaddexp(forward[t][next_state], new_prob)

                        # Propagar a través de más estados Delete
                        stack = [(next_state, new_prob)]
                        processed = set()

                        while stack:
                            d_state, d_prob = stack.pop()
                            if d_state in processed:
                                continue
                            processed.add(d_state)

                            for d_next in states:
                                if d_state in trans_probs and d_next in trans_probs[d_state]:
                                    if d_next.startswith('D'):
                                        d_new_prob = d_prob + log_trans[d_state][d_next]
                                        if d_new_prob > forward[t][d_next]:
                                            forward[t][d_next] = d_new_prob
                                            stack.append((d_next, d_new_prob))
                                    elif d_next.startswith('M') or d_next.startswith('I'):
                                        if symbol in log_emit.get(d_next, {}):
                                            d_new_prob = d_prob + log_trans[d_state][d_next] + log_emit[d_next][symbol]
                                            forward[t+1][d_next] = np.logaddexp(forward[t+1][d_next], d_new_prob)

    # Calcular probabilidad total - suma sobre todas las posibles finalizaciones
    final_prob = -np.inf
    for t in range(T + 1):
        for state in states:
            if 'E' in trans_probs.get(state, {}):
                if forward[t][state] > -np.inf:
                    path_prob = forward[t][state] + log_trans[state]['E']
                    final_prob = np.logaddexp(final_prob, path_prob)

    # Incluir una normalización por la longitud
    normalized_log_prob = final_prob / T if T > 0 else final_prob

    return normalized_log_prob

def backward_algorithm(sequence: str,
                      trans_probs: Dict[str, Dict[str, float]],
                      emit_probs: Dict[str, Dict[str, float]],
                      states: List[str],
                      amino_acids: List[str]) -> List[Dict[str, float]]:
    """
    Implementa el algoritmo Backward para HMM con manejo de estados Delete

    Args:
        sequence: Secuencia de aminoácidos
        trans_probs: Matriz de probabilidades de transición
        emit_probs: Matriz de probabilidades de emisión
        states: Lista de estados del modelo
        amino_acids: Lista de aminoácidos

    Returns:
        Lista de diccionarios con valores backward para cada posición y estado
    """
    # Limpiar secuencia de caracteres no estándar
    cleaned_sequence = ''.join(ch if ch in amino_acids else amino_acids[0] for ch in sequence)
    sequence = cleaned_sequence

    # Convertir a logaritmos para evitar underflow
    log_trans = {s: {d: np.log(p) if p > 0 else -np.inf
                    for d, p in dests.items()}
                for s, dests in trans_probs.items()}

    log_emit = {s: {a: np.log(p) if p > 0 else -np.inf
                   for a, p in aa_probs.items()}
               for s, aa_probs in emit_probs.items()}

    # Inicializar matriz backward
    T = len(sequence)
    backward = [{state: -np.inf for state in states} for _ in range(T + 1)]

    # Inicializar estado final
    for state in states:
        if 'E' in trans_probs.get(state, {}):
            backward[T][state] = log_trans[state]['E']

    # Función recursiva para procesar estados Delete
    def process_delete_backward(t: int, current_state: str):
        for prev_state in states:
            if current_state in trans_probs.get(prev_state, {}):
                if prev_state.startswith('D'):
                    # Estados Delete no emiten símbolos
                    new_prob = log_trans[prev_state][current_state] + backward[t][current_state]
                    backward[t][prev_state] = np.logaddexp(backward[t][prev_state], new_prob)
                    # Recursivamente procesar más estados Delete
                    process_delete_backward(t, prev_state)

    # Calculamos el backward para cada posición de la secuencia, de atrás hacia adelante
    for t in range(T - 1, -1, -1):
        # Procesar nodos que emiten en esta posición
        for next_state in states:
            if next_state.startswith('M') or next_state.startswith('I'):
                if t < len(sequence):
                    symbol = sequence[t]
                    emit_val = log_emit[next_state].get(symbol, -np.inf)
                    for prev_state in states:
                        if next_state in trans_probs.get(prev_state, {}):
                            trans_val = log_trans[prev_state][next_state]
                            back_val = backward[t+1][next_state]
                            if trans_val != -np.inf and emit_val != -np.inf and back_val != -np.inf:
                                new_prob = trans_val + emit_val + back_val
                                backward[t][prev_state] = np.logaddexp(backward[t][prev_state], new_prob)

        # Procesar nodos Delete recursivamente
        for state in states:
            if state.startswith('D'):
                process_delete_backward(t, state)

        # Procesar nodos que finalizan
        if t < T:
            for state in states:
                if 'E' in trans_probs.get(state, {}):
                    new_prob = log_trans[state]['E']
                    backward[t][state] = np.logaddexp(backward[t][state], new_prob)

    return backward

def baum_welch(sequences: List[str],
               initial_trans_probs: Dict[str, Dict[str, float]],
               initial_emit_probs: Dict[str, Dict[str, float]],
               states: List[str],
               amino_acids: List[str],
               max_iter: int = 10,
               tol: float = 1e-4) -> Tuple[Dict[str, Dict[str, float]], Dict[str, Dict[str, float]]]:
    """
    Implementa el algoritmo de Baum-Welch para entrenar un HMM
    con manejo adecuado de estados Delete y estabilidad numérica

    Args:
        sequences: Lista de secuencias de aminoácidos
        initial_trans_probs: Matriz inicial de probabilidades de transición
        initial_emit_probs: Matriz inicial de probabilidades de emisión
        states: Lista de estados del modelo
        amino_acids: Lista de aminoácidos
        max_iter: Número máximo de iteraciones
        tol: Tolerancia para la convergencia

    Returns:
        trans_probs, emit_probs: Matrices optimizadas de transición y emisión
    """
    trans_probs = initial_trans_probs
    emit_probs = initial_emit_probs

    # Convertir a logaritmos para cálculos internos
    log_likelihood_old = -np.inf

    for iteration in range(max_iter):
        print(f"Iteración Baum-Welch {iteration+1}/{max_iter}")

        # Contadores para re-estimar parámetros
        trans_counts = defaultdict(lambda: defaultdict(float))
        emit_counts = defaultdict(lambda: defaultdict(float))
        state_counts = defaultdict(float)

        log_likelihood_new = 0.0

        # Procesar cada secuencia
        for seq_idx, sequence in enumerate(sequences):
            if seq_idx % 10 == 0:
                print(f"  Procesando secuencia {seq_idx+1}/{len(sequences)} ({(seq_idx+1)/len(sequences)*100:.1f}%)")

            # Limpiar secuencia
            cleaned_sequence = ''.join(ch if ch in amino_acids else amino_acids[0] for ch in sequence)
            sequence = cleaned_sequence

            try:
                # Calcular matrices forward y backward
                # Usamos implementaciones que manejan estados Delete correctamente
                forward_matrices = []
                backward_matrices = []

                # Inicializar la matriz forward
                T = len(sequence)
                forward = [{state: -np.inf for state in states} for _ in range(T + 1)]
                forward[0]['S'] = 0.0  # log(1)

                # Implementar forward_calc similar a forward_algorithm
                # ... (código de forward y backward omitido por brevedad)

                # En lugar de recalcular, usamos las funciones existentes
                forward_val = forward_algorithm_fixed(sequence, trans_probs, emit_probs, states, amino_acids)
                backward = backward_algorithm(sequence, trans_probs, emit_probs, states, amino_acids)

                # Actualizar likelihood
                log_likelihood_new += forward_val

                # Cálculo de gamma (probabilidad de estar en cada estado)
                for t in range(T + 1):
                    for state in states:
                        for next_state in trans_probs.get(state, {}):
                            # Calcular xi (probabilidad de transición)
                            if t < T:
                                if next_state.startswith('M') or next_state.startswith('I'):
                                    # Estados que emiten símbolos
                                    symbol = sequence[t]
                                    if forward[t][state] > -np.inf and backward[t+1][next_state] > -np.inf:
                                        trans_val = np.log(trans_probs[state][next_state])
                                        emit_val = np.log(emit_probs[next_state].get(symbol, 1.0/len(amino_acids)))
                                        xi_val = forward[t][state] + trans_val + emit_val + backward[t+1][next_state] - forward_val

                                        # Convertir de log-space
                                        xi_prob = np.exp(xi_val)
                                        trans_counts[state][next_state] += xi_prob
                                        state_counts[state] += xi_prob

                                        # Acumular emisiones
                                        emit_counts[next_state][symbol] += xi_prob

                                elif next_state.startswith('D') or next_state == 'E':
                                    # Estados que no emiten símbolos
                                    if forward[t][state] > -np.inf and backward[t][next_state] > -np.inf:
                                        trans_val = np.log(trans_probs[state][next_state])
                                        xi_val = forward[t][state] + trans_val + backward[t][next_state] - forward_val

                                        # Convertir de log-space
                                        xi_prob = np.exp(xi_val)
                                        trans_counts[state][next_state] += xi_prob
                                        state_counts[state] += xi_prob

            except Exception as e:
                print(f"Error en secuencia {seq_idx}: {e}")
                continue

        # Re-estimar parámetros
        new_trans_probs = {}
        for state in states:
            new_trans_probs[state] = {}
            total = state_counts.get(state, 0.0)

            if total > 0:
                for next_state in trans_probs.get(state, {}):
                    count = trans_counts.get(state, {}).get(next_state, 0.0)
                    new_trans_probs[state][next_state] = count / total
            else:
                # Si no hay conteos, mantener valores anteriores
                for next_state in trans_probs.get(state, {}):
                    new_trans_probs[state][next_state] = trans_probs[state][next_state]

        # Re-estimar probabilidades de emisión
        new_emit_probs = {}
        for state in states:
            if state.startswith('M') or state.startswith('I'):
                new_emit_probs[state] = {}
                total = sum(emit_counts.get(state, {}).values())

                if total > 0:
                    for symbol in amino_acids:
                        count = emit_counts.get(state, {}).get(symbol, 0.0)
                        new_emit_probs[state][symbol] = count / total
                else:
                    # Si no hay conteos, mantener valores anteriores
                    for symbol in amino_acids:
                        new_emit_probs[state][symbol] = emit_probs.get(state, {}).get(symbol, 1.0/len(amino_acids))

        # Verificar convergencia
        print(f"  Log-likelihood: {log_likelihood_new}")
        if abs(log_likelihood_new - log_likelihood_old) < tol:
            print(f"Convergencia alcanzada en la iteración {iteration+1}")
            break

        log_likelihood_old = log_likelihood_new
        trans_probs = new_trans_probs
        emit_probs = new_emit_probs

    return trans_probs, emit_probs

def train_hmm_with_baum_welch(sequences: List[str],
                             initial_trans_probs: Dict[str, Dict[str, float]],
                             initial_emit_probs: Dict[str, Dict[str, float]],
                             states: List[str],
                             amino_acids: List[str],
                             max_iter: int = 10) -> Tuple[Dict[str, Dict[str, float]], Dict[str, Dict[str, float]], List[str], List[str]]:
    """
    Entrenar HMM usando el algoritmo de Baum-Welch mejorado

    Args:
        sequences: Lista de secuencias de aminoácidos
        initial_trans_probs: Matriz inicial de probabilidades de transición
        initial_emit_probs: Matriz inicial de probabilidades de emisión
        states: Lista de estados del modelo
        amino_acids: Lista de aminoácidos
        max_iter: Número máximo de iteraciones

    Returns:
        Matrices optimizadas de transición y emisión, además de listas de estados y aminoácidos
    """
    print(f"Iniciando entrenamiento Baum-Welch mejorado con {len(sequences)} secuencias")

    # Filtrar secuencias muy cortas o con caracteres no válidos
    valid_sequences = []
    for seq in sequences:
        cleaned_seq = ''.join(ch if ch in "ACDEFGHIKLMNPQRSTVWY-" else '-' for ch in seq)
        if len(cleaned_seq.replace('-', '')) >= 5:  # Al menos 5 aminoácidos válidos
            valid_sequences.append(cleaned_seq)

    if len(valid_sequences) < len(sequences):
        print(f"Se filtraron {len(sequences) - len(valid_sequences)} secuencias no válidas")

    if not valid_sequences:
        raise ValueError("No quedan secuencias válidas para entrenamiento")

    # Ejecutar Baum-Welch mejorado
    optimized_trans_probs, optimized_emit_probs = baum_welch(
        sequences=valid_sequences,
        initial_trans_probs=initial_trans_probs,
        initial_emit_probs=initial_emit_probs,
        states=states,
        amino_acids=amino_acids,
        max_iter=max_iter
    )

    # Verificar que no haya probabilidades inválidas
    for state in optimized_trans_probs:
        for next_state in optimized_trans_probs[state]:
            if not np.isfinite(optimized_trans_probs[state][next_state]):
                print(f"⚠️ Corrigiendo transición no válida: {state} -> {next_state}")
                optimized_trans_probs[state][next_state] = 0.001

    for state in optimized_emit_probs:
        for aa in optimized_emit_probs[state]:
            if not np.isfinite(optimized_emit_probs[state][aa]):
                print(f"⚠️ Corrigiendo emisión no válida: {state} -> {aa}")
                optimized_emit_probs[state][aa] = 1.0 / len(amino_acids)

    return optimized_trans_probs, optimized_emit_probs, amino_acids, states

In [8]:
import pickle
import numpy as np
from pathlib import Path
from typing import Dict, Tuple, List, Optional, Union

class ConotoxinPredictor:
    def __init__(self, model_path: Optional[str] = None):
        # Inicializar todos los atributos
        self.trans_probs: Dict = {}
        self.emit_probs: Dict = {}
        self.amino_acids: List[str] = []
        self.states: List[str] = []
        self.threshold: float = 0.0
        self.sequences: List[str] = []

        if model_path:
            self.load_model(model_path)

    def train(self, fasta_path: str, save_path: Optional[str] = None):
        """Entrena un nuevo modelo"""
        print("Cargando secuencias...")
        self.sequences = parse_fasta(fasta_path)

        # Preprocesar el alineamiento
        print("Preprocesando alineamiento...")
        self.sequences = filter_high_gap_columns(self.sequences, max_gap_ratio=0.7)

        print(f"Construyendo HMM con {len(self.sequences)} secuencias...")
        self.trans_probs, self.emit_probs, self.amino_acids, self.states = build_hmm(self.sequences)

        # Refinar con Baum-Welch (usar un subconjunto para acelerar)
        print("Refinando modelo con Baum-Welch...")
        sample_size = min(100, len(self.sequences))
        sample_seqs = self.sequences[:sample_size]
        self.trans_probs, self.emit_probs, self.amino_acids, self.states = train_hmm_with_baum_welch(
            sample_seqs, self.trans_probs, self.emit_probs, self.states, self.amino_acids, max_iter=3
        )

        # Calcular el threshold después de entrenar
        print("Calculando threshold...")
        self.threshold = self._calculate_threshold()

        if save_path:
            print(f"Guardando modelo en {save_path}...")
            self.save_model(save_path)

    def _calculate_threshold(self, sample_size: int = 20, percentile: float = 10.0) -> float:
        """Calcula el threshold con más variabilidad en muestras"""

        if not self.sequences:
            raise ValueError("No hay secuencias para calcular el threshold")

        # Limitar el tamaño de muestra
        sample_size = min(sample_size, len(self.sequences))
        scores = []

        # Usar secuencias más diversas (no solo aleatorias)
        import random
        from collections import Counter

        # Calcular la longitud de secuencias sin gaps
        seq_lengths = [len(seq.replace('-', '')) for seq in self.sequences]

        # Tomar muestras de diferentes longitudes para mayor diversidad
        samples = []
        length_groups = {}
        for i, seq in enumerate(self.sequences):
            length = seq_lengths[i]
            if length not in length_groups:
                length_groups[length] = []
            length_groups[length].append(i)

        # Seleccionar de diferentes grupos
        for length, indices in sorted(length_groups.items()):
            if indices and len(samples) < sample_size:
                samples.append(random.choice(indices))

        # Completar con aleatorios si es necesario
        while len(samples) < sample_size:
            idx = random.randint(0, len(self.sequences) - 1)
            if idx not in samples:
                samples.append(idx)

        # Calcular scores
        for i, idx in enumerate(samples):
            seq = self.sequences[idx]
            print(f"Calculando score {i+1}/{sample_size} ({(i+1)/sample_size*100:.1f}%)")

            # Limpiar y recortar si es necesario
            cleaned_seq = ''.join(ch if ch in self.amino_acids else self.amino_acids[0] for ch in seq)

            # Usar el algoritmo Forward mejorado
            score = forward_algorithm_fixed(cleaned_seq,
                                    self.trans_probs,
                                    self.emit_probs,
                                    self.states,
                                    self.amino_acids)

            scores.append(score)
            print(f"  Seq length: {len(cleaned_seq)}, Score: {score}")

        # Imprimir estadísticas
        print(f"Estadísticas de scores: Min={min(scores)}, Max={max(scores)}, Avg={sum(scores)/len(scores)}")

        # Usar el percentil especificado como threshold
        threshold = float(np.percentile(scores, percentile))
        print(f"Threshold calculado: {threshold}")
        return threshold

    def predict(self, sequence: str) -> Tuple[bool, float]:
        """Predice si una secuencia es conotoxina"""
        if not hasattr(self, 'threshold') or self.threshold is None:
            raise ValueError("Modelo no entrenado: falta el threshold")

        # Limpiar la secuencia de caracteres no estándar


        # Usar algoritmo Forward en lugar de Viterbi
        score = forward_algorithm_fixed(sequence,
                                 self.trans_probs,
                                 self.emit_probs,
                                 self.states,
                                 self.amino_acids)
        return score > self.threshold, score

    def save_model(self, path: str):
        """Guarda el modelo en disco"""
        if not hasattr(self, 'threshold'):
            raise ValueError("Modelo no entrenado: no se puede guardar")

        model_data = {
            'trans_probs': self.trans_probs,
            'emit_probs': self.emit_probs,
            'amino_acids': self.amino_acids,
            'states': self.states,
            'threshold': self.threshold
        }

        # Crear directorio si no existe
        Path(path).parent.mkdir(parents=True, exist_ok=True)

        with open(path, 'wb') as f:
            pickle.dump(model_data, f)

    def load_model(self, path: str):
        """Carga un modelo desde disco"""
        with open(path, 'rb') as f:
            model_data = pickle.load(f)

        self.trans_probs = model_data['trans_probs']
        self.emit_probs = model_data['emit_probs']
        self.amino_acids = model_data['amino_acids']
        self.states = model_data['states']
        self.threshold = model_data['threshold']

# Train

In [ ]:
import argparse
import os
import sys
from pathlib import Path

def main():
    input_fasta = "/content/drive/MyDrive/Proyecto/Cluster95_alineamiento.fasta"
    output_model = "/content/drive/MyDrive/Proyecto/conotoxins.pkl"
    iterations = 100

    # parser = argparse.ArgumentParser(description='Entrena un modelo HMM para conotoxinas')
    # parser.add_argument('input_fasta', help='Archivo FASTA con secuencias alineadas')
    # parser.add_argument('output_model', help='Archivo de salida para el modelo entrenado')
    # parser.add_argument('--iterations', type=int, default=5,
    #                     help='Número de iteraciones para el algoritmo de Baum-Welch')
    # args = parser.parse_args()

    # Verificar que el archivo FASTA existe
    # if not os.path.isfile(args.input_fasta):
    #     sys.exit(f"Error: El archivo FASTA '{args.input_fasta}' no existe.")

    # # Crear directorio de salida si no existe
    # output_dir = os.path.dirname(args.output_model)
    # if output_dir:
    #     os.makedirs(output_dir, exist_ok=True)

    try:
        print("Entrenando modelo...")
        predictor = ConotoxinPredictor()
        predictor.train(input_fasta, output_model)
        print(f"Modelo entrenado guardado en {output_model}")
    except Exception as e:
        sys.exit(f"Error durante el entrenamiento: {e}")

if __name__ == '__main__':
    main()

Entrenando modelo...
Cargando secuencias...
Se leyeron 2523 secuencias del archivo FASTA
Preprocesando alineamiento...
Construyendo HMM con 2523 secuencias...
⚠️ Advertencia: Se encontraron aminoácidos no estándar: {'B'}
   Estos serán tratados como caracteres normales
Contando transiciones y emisiones...
Procesando secuencia 0/2523 (0.0%)
Procesando secuencia 100/2523 (4.0%)
Procesando secuencia 200/2523 (7.9%)
Procesando secuencia 300/2523 (11.9%)
Procesando secuencia 400/2523 (15.9%)
Procesando secuencia 500/2523 (19.8%)
Procesando secuencia 600/2523 (23.8%)
Procesando secuencia 700/2523 (27.7%)
Procesando secuencia 800/2523 (31.7%)
Procesando secuencia 900/2523 (35.7%)
Procesando secuencia 1000/2523 (39.6%)
⚠️ Secuencia 1008 tiene caracteres no válidos: {'B'}
Procesando secuencia 1100/2523 (43.6%)
Procesando secuencia 1200/2523 (47.6%)
Procesando secuencia 1300/2523 (51.5%)
Procesando secuencia 1400/2523 (55.5%)
Procesando secuencia 1500/2523 (59.5%)
Procesando secuencia 1600/2523 

# Predict

In [ ]:
import argparse
import os
import sys
from pathlib import Path
import re

def is_valid_sequence(seq):
    """Verifica si una secuencia contiene solo aminoácidos válidos"""
    return bool(re.match(r'^[ACDEFGHIKLMNPQRSTVWY]+$', seq, re.I))

def process_sequence(seq):
    """Limpia y formatea una secuencia de aminoácidos"""
    # Eliminar espacios y caracteres no válidos
    clean_seq = re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', seq.upper())
    return clean_seq

def main():
    model_path = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/conotoxins.pkl"
    fasta = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/proteoma.fasta"
    sequences = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/"
    output = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/salida.txt"

    # parser = argparse.ArgumentParser(description='Predice si secuencias son conotoxinas')
    # parser.add_argument('model_path', help='Archivo del modelo entrenado')
    # parser.add_argument('--fasta', help='Archivo FASTA con secuencias a evaluar')
    # parser.add_argument('--sequences', nargs='+', help='Secuencias a evaluar directamente')
    # parser.add_argument('--output', help='Archivo para guardar resultados (opcional)')
    # args = parser.parse_args()



    # Verificar que se especificó al menos una fuente de secuencias
    if not fasta and not sequences:
        sys.exit("Error: Debe proporcionar secuencias mediante --fasta o --sequences")

    # Verificar que el modelo existe
    if not os.path.isfile(model_path):
        sys.exit(f"Error: El archivo de modelo '{model_path}' no existe")

    try:
        print("Cargando modelo...")
        predictor = ConotoxinPredictor(model_path)

        sequences_to_process = []

        # Procesar secuencias de FASTA si se proporciona
        if fasta:
            try:
                fasta_sequences = parse_fasta(fasta)
                sequences_to_process.extend([(f"FASTA_{i+1}", seq) for i, seq in enumerate(fasta_sequences)])
            except Exception as e:
                sys.exit(f"Error al leer archivo FASTA: {e}")

        # Procesar secuencias pasadas directamente
        if sequences:
            sequences_to_process.extend([(f"SEQ_{i+1}", process_sequence(seq))
                                        for i, seq in enumerate(sequences)])

        # Preparar archivo de salida si se especificó
        out_file = None
        if output:
            try:
                out_file = open(output, 'w')
                out_file.write("Secuencia\tEs_conotoxina\tScore\n")
            except Exception as e:
                sys.exit(f"Error al abrir archivo de salida: {e}")

        # Procesar cada secuencia
        results = []
        for seq_id, seq in sequences_to_process:
            if not seq:  # Ignorar secuencias vacías
                continue

            print(f"Analizando {seq_id}: {seq[:50]}...")
            try:
                is_conotoxin, score = predictor.predict(seq)
                result = (seq_id, seq[:50] + "..." if len(seq) > 50 else seq, is_conotoxin, score)
                results.append(result)

                print(f"  Es conotoxina: {is_conotoxin}")
                print(f"  Score: {score:.2f}\n")

                # Guardar en archivo si es necesario
                if out_file:
                    out_file.write(f"{seq_id}\t{is_conotoxin}\t{score:.6f}\n")
            except Exception as e:
                print(f"Error al analizar {seq_id}: {e}\n")

        # Cerrar archivo de salida
        if out_file:
            out_file.close()
            print(f"Resultados guardados en {output}")

        # Resumen final
        conotoxins = sum(1 for _, _, is_cono, _ in results if is_cono)
        print(f"\nResumen: {conotoxins} de {len(results)} secuencias clasificadas como conotoxinas")

    except Exception as e:
        sys.exit(f"Error durante la predicción: {e}")

if __name__ == '__main__':
    main()

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  Es conotoxina: True
  Score: 0.29

Analizando FASTA_7075: LRFFILAFIVYVNKFTNSLFCRVSPYPNSYHHCLIAPSIASKMLLELGFL...
  Es conotoxina: True
  Score: 0.21

Analizando FASTA_7076: PISTHILLEGAVTKPWLSVKLQLTLIEITNCKKNNNNKLSILCVELPTEH...
  Es conotoxina: True
  Score: 0.21

Analizando FASTA_7077: VSTLISESLNSTPANLKTWLKKNQKNKNKQVRTVSTEAVKI...
  Es conotoxina: True
  Score: 0.31

Analizando FASTA_7078: VILVLAAGTVVATSACALVHMNIALRILIVGSAAKPTPTNVVLFRIMSLP...
  Es conotoxina: True
  Score: 0.22

Analizando FASTA_7079: RRWVGLHCEIVVKQVHMLLGPGMLGDSPSYLGLLCLHNNKVDW...
  Es conotoxina: True
  Score: 0.30

Analizando FASTA_7080: PETVPSVRFAHRRPHIQPIASTPLLTQTNSVQKELNRSLDYPRIFQVKPT...
  Es conotoxina: False
  Score: 0.16

Analizando FASTA_7081: NVIITILYNICKITGFFYALICIRLESLTILFLYCKNLGTPFCICSLCYI...
  Es conotoxina: True
  Score: 0.26

Analizando FASTA_7082: QAFRVFLSSLGGQWRTSQIMKEQTEKKCTSPTLLCTRNGTTALATKLFIS...
  Es conotoxina: True
  Score: 0.26


# Transcriptoma a proteoma

In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def get_longest_orf(dna_seq):
    """
    Toma una secuencia de ADN (o ARN con T en lugar de U) y devuelve el ORF más largo.
    Se consideran los 3 marcos de lectura en la hebra directa.
    """
    # Convertir a mayúsculas y reemplazar U si existiera (para trabajar con ADN)
    seq = str(dna_seq).upper().replace("U", "T")
    longest_protein = ""

    # Para cada marco de lectura
    for frame in range(3):
        # Traducir toda la secuencia desde ese marco sin detenerse en el codón de parada
        translated_seq = Seq(seq[frame:]).translate(to_stop=False)
        # Dividir la traducción en segmentos (ORFs) usando el codón de parada "*"
        for protein in translated_seq.split("*"):
            if len(protein) > len(longest_protein):
                longest_protein = protein
    return longest_protein

# Archivo de entrada y salida (por ejemplo, el archivo que has proporcionado lo llamamos '2.txt')
input_file = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/transcriptoma_dietas.txt"
output_file = "/content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/proteoma.fasta"

proteins = []

# Lectura y procesamiento de cada entrada del FASTA
for record in SeqIO.parse(input_file, "fasta"):
    orf_protein = get_longest_orf(record.seq)
    if orf_protein:
        prot_record = SeqRecord(
            Seq(str(orf_protein)),
            id=record.id,
            description="Traducción del ORF más largo"
        )
        proteins.append(prot_record)

# Escribir las secuencias traducidas en un nuevo archivo FASTA
SeqIO.write(proteins, output_file, "fasta")
print(f"Se han guardado {len(proteins)} proteínas en {output_file}")


/usr/local/lib/python3.11/dist-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Se han guardado 108212 proteínas en /content/drive/MyDrive/Algoritmos para bioinformatica/Proyecto/proteoma.fasta
